# Multiple linear regression - Statistical model selection
## Fitting a multiple linear regression on sales of carseats and introducing key concepts for dealing with categorical data and model selection methodology.
## Model selection will be based on statistical significance

### **1. Importing libraries and loading data**

In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from ISLP import load_data

from ISLP.models import (ModelSpec as MS, summarize, poly)

In [5]:
Carseats = load_data('Carseats')
Carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
0,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
1,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
2,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
3,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
4,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [7]:
Carseats.shape

(400, 11)

### For the purpose of this exercise assume the relevant steps have been taken in preparing and processing the data and ensuring that it is clean and usable. For references check other projects in the github repository.
### This dataset contains 400 entries and 11 features

### **2. Fitting a multiple linear regression**

### Fitting a multiple linear regression to predict sales using 10 predictors containing qualitative and quantitative data.
### Urban indicates whether sales took place in an urban area and US indicates if the transaction was made in the US
### Using a **backward model selection**, we start with all predictors and remove any features that are statistically insignificant using the p-value.  

In [8]:
# Seperating target variable and predictors and using modelspec() to create a model matrix
predictors = list(Carseats.columns.drop(['Sales']))
X = MS(predictors).fit_transform(Carseats)
y = Carseats['Sales']

# Fitting a linear regression
model = sm.OLS(y, X)
rslt = model.fit()
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     243.4
Date:                Fri, 04 Aug 2023   Prob (F-statistic):          1.60e-166
Time:                        17:26:58   Log-Likelihood:                -568.99
No. Observations:                 400   AIC:                             1162.
Df Residuals:                     388   BIC:                             1210.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             5.6606      0.603      9.380      0.000       4.474       6.847
CompPrice             0.0928      0.004     22.378      0.000       0.085       0.101
Income                0.0158      0.002      8.565      0.000       0.012       0.019
Advertising           0.1231      0.011     11.066      0.000       0.101       0.145
Population            0.0002      0.000      0.561      0.575      -0.001       0.001
Price                -0.0954      0.003    -35.700      0.000      -0.101      -0.090
ShelveLoc[Good]       4.8502      0.153     31.678      0.000       4.549       5.151
ShelveLoc[Medium]     1.9567      0.126     15.516      0.000       1.709       2.205
Age                  -0.0460      0.003    -14.472      0.000      -0.052      -0.040
Education            -0.0211      0.020     -1.070      0.285      -0.060       0.018
Urban[Yes]            0.1229      0.113      1.088      0.277      -0.099       0.345
US[Yes]              -0.1841      0.150     -1.229      0.220      -0.479       0.111
==============================================================================
Omnibus:                        0.811   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.667   Jarque-Bera (JB):                0.765
Skew:                           0.107   Prob(JB):                        0.682
Kurtosis:                       2.994   Cond. No.                     4.15e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.15e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Using the modelspec(), the model-matrix builder has created a `ShelveLoc[Good]` dummy variable that takes on a value of 1 if the shelving location is good, and 0 otherwise and has also created a `ShelveLoc[Medium]` dummy variable that equals 1 if the shelving location is medium, and 0 otherwise, lastly a bad shelving location corresponds zero for each of the two dummy variables. This process is repeated for Urban and US feature which only has 2 levels in the dummy variable. **This an example of one hot coding, a necessary step in processing categorical data for regression analysis**.
### Using the p-values, **we can reject the null hypotheses** (which states that the coefficients are 0) for predictors that have a **p-value less than 0.05** which leaves 6 of our preditors except Population, Education, Urban, and US. This is to say that the population, education level and location is statistically insignificant in determining sales of car seats.
### Using this information we can further reduce our model by removing the insignficant features and compare its results with our full model. This is the process of backward model selection.

In [9]:
reduced_predictors = list(Carseats.columns.drop(['Sales','Population', 'Education', 'Urban', 'US']))
X = MS(reduced_predictors).fit_transform(Carseats)
model = sm.OLS(y, X)
rslt = model.fit()
rslt.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.872
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     381.4
Date:                Fri, 04 Aug 2023   Prob (F-statistic):          1.25e-170
Time:                        17:49:15   Log-Likelihood:                -571.24
No. Observations:                 400   AIC:                             1158.
Df Residuals:                     392   BIC:                             1190.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             5.4752      0.505     10.842      0.000       4.482       6.468
CompPrice             0.0926      0.004     22.451      0.000       0.084       0.101
Income                0.0158      0.002      8.590      0.000       0.012       0.019
Advertising           0.1159      0.008     15.006      0.000       0.101       0.131
Price                -0.0953      0.003    -35.699      0.000      -0.101      -0.090
ShelveLoc[Good]       4.8357      0.152     31.710      0.000       4.536       5.135
ShelveLoc[Medium]     1.9520      0.125     15.569      0.000       1.706       2.198
Age                  -0.0461      0.003    -14.521      0.000      -0.052      -0.040
==============================================================================
Omnibus:                        0.766   Durbin-Watson:                   1.988
Prob(Omnibus):                  0.682   Jarque-Bera (JB):                0.810
Skew:                           0.104   Prob(JB):                        0.667
Kurtosis:                       2.929   Cond. No.                     1.91e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.91e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### To compare how each model performs we look at the R-sqaured value. This is a measure of how much the selected model is better fitted to explain changes in sales. **A high R-sqaured value indicates a good fit**. 
### Seeing that **both models have an R-sqaured of 87%** (both models explains changes in sales equally well), we can **compare the F-statistic**. The smaller model has an F-statistic of 381.4 which is significantly more than the F-statistic of the larger model 243.4, we can conclude that the smaller model is the best model to explain changes in sales and predicting future sales as it is more statistically significant.
### Other reasons to select a smaller model is to avoid overfitting the data, helps deal with effects of collinearity, and fewer predictors increases interpretability and clarity in the model. 
### **Using the smaller model we can extract useful insights:**
* ### The price of a car seat has a negative relationship with sales (negative coef e.g -0.0953). This is to say that for every dollar increase in price, sales will drop by 9.53%.
* ### A good shelving location is associated with higher sales as compared to a bad shelving location.
* ### The coefficient of a medium shelving location is positive but lower than a good shelving location. This is to say that although a medium shelving location increases sales more than a bad location, it is still lower than a good shelving location. 
* ### The same logic can be applied in interpreting the impact of each features coef